In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
import re

# Set up LaTeX rendering options for matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'  # Set font for LaTeX rendering
latex_image_dir = "latex_eqn_images"  # Directory to store generated images of LaTeX equations
os.makedirs(latex_image_dir, exist_ok=True)  # Ensure directory exists

# Load data from CSV file and select the last 5 rows
data = pd.read_csv("QUE4.csv")
# df = data.tail(5)

# Function to render LaTeX string as an image file
def render_latex_to_image(latex_string, filename, img_width=50):
    # Create a figure for rendering the LaTeX string
    fig = plt.figure(figsize=(img_width/100, 0.5))
    fig.text(x=0.5, y=0.5, s=latex_string, fontsize=11, ha='center', va='center')
    # Save the figure as an image with high DPI for clarity
    fig.savefig(filename, dpi=500, pad_inches=0.03, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free up resources

# Function to wrap text into lines that fit within a specified width
def wrap_text(text, max_width):
    # Split the text into parts, separating LaTeX equations and regular text
    words = re.split(r'(\$.*?\$)', text)
    lines = []
    current_line = ""

    for word in words:
        # If word is a LaTeX equation, add it as a separate line
        if re.match(r'\$.*?\$', word):
            if current_line:  # Append the current line if it contains text
                lines.append(current_line.strip())
                current_line = ""
            lines.append(word)  # Add the LaTeX equation as its own line
        else:  # If word is regular text, break it down into sub-words
            for sub_word in word.split():
                # Check if adding this word to the current line fits within max width
                if stringWidth(current_line + " " + sub_word, "Helvetica", 10) < max_width:
                    current_line += " " + sub_word
                else:
                    # If it doesn't fit, add the current line and start a new line
                    lines.append(current_line.strip())
                    current_line = sub_word  # Start new line with current word
    if current_line:  # Add any remaining text in current_line to lines
        lines.append(current_line.strip())
    
    return lines

# Function to generate a PDF with LaTeX-rendered images and wrapped text
def create_pdf_with_latex(df, filename="output_fixed_wrap_text_and_LaTeX_Eqn.pdf"):
    c = canvas.Canvas(filename, pagesize=landscape(letter))  # Create PDF canvas
    width, height = landscape(letter)  # Get page dimensions

    # Set layout margins and spacing parameters
    left_margin = 50
    top_margin = height - 50
    line_spacing = 30
    max_line_width = 575
    min_y_position = 50  # Minimum Y position to avoid text running off the page

    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        y_position = top_margin  # Reset Y position for each row

        # Loop through each column and value in the row
        for col, value in row.items():
            text = f"{col}: {value}"  # Create text string for column and value
            lines = wrap_text(text, max_line_width)  # Wrap text to fit within max width
            x_position = left_margin  # Reset X position for each row

            # Loop through each wrapped line
            for line in lines:
                # Ensure y_position stays within the printable area
                if y_position <= min_y_position:
                    c.showPage()  # Start a new page if current Y position is too low
                    y_position = top_margin
                    x_position = left_margin

                if re.match(r'\$.*?\$', line):  # If the line contains a LaTeX equation
                    # Render LaTeX to image if not already done
                    latex_image_path = os.path.join(latex_image_dir, f"latex_{index}.png")
                    if not os.path.exists(latex_image_path):
                        render_latex_to_image(line, latex_image_path, img_width=70)

                    image_width = 70  # Define image width
                    if x_position + image_width <= max_line_width:
                        # Draw the image on the current line if it fits
                        c.drawImage(latex_image_path, x_position + 40, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 45
                    else:
                        # Move to the next line if image doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawImage(latex_image_path, x_position, y_position - 10, width=image_width, height=30)
                        x_position += image_width + 5

                else:
                    # For regular text, measure the line width and position it accordingly
                    line_width = stringWidth(line, "Helvetica", 10)
                    if x_position + line_width <= max_line_width:
                        # Draw the text on the same line if it fits
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5
                    else:
                        # Move to the next line if text doesn't fit
                        y_position -= line_spacing
                        x_position = left_margin
                        c.drawString(x_position, y_position, line)
                        x_position += line_width + 5

            y_position -= line_spacing  # Add extra space after each row
        c.showPage()  # Move to a new page after each row

    c.save()  # Save and close the PDF file

# Run the PDF generation function
create_pdf_with_latex(data)
